In [1]:
import urllib
import re
from bs4 import SoupStrainer 
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd 
import time 


In [2]:

def find_jobs_and_location(url,position,location):
    #get URL
    global driver
    driver = webdriver.Chrome()
    baseURL = "https://www.glassdoor.com/index.htm"
    driver.get(baseURL)
    driver.implicitly_wait(3)
    # Search job and location
    keywordElement = driver.find_element_by_id("KeywordSearch")
    locationElement = driver.find_element_by_id("LocationSearch")
    keywordElement.clear()
    keywordElement.send_keys("Data Analyst")
    locationElement.clear()
    locationElement.send_keys("New York, NY")
    driver.find_element_by_class_name("gd-btn-mkt").click()

find_jobs_and_location("https://www.glassdoor.com/index.htm","Data Analyst","New York, NY")    

In [3]:
# get text page by page 
page_remaining = True
page =[]
while page_remaining:
    time.sleep(3)
    try:       
        emailMeJob_pop = driver.find_element_by_class_name("mfp-close")
        emailMeJob_pop.click()
    except:
        current_html = driver.page_source
        bs = BeautifulSoup(current_html,"html.parser")   
        #scrape company name,location,salary, and company star review 
        jobs_title =bs.select("td.job_title")
        companies_name = bs.select("td.company")
        jobs_location = bs.select("td.location")
        jobs_salary = bs.select("td.salary")
        companies_star = bs.select("span.compactStars")
        
        #find 30 joblinks under tag<td class="job_title">, if use find_all(a class_="jobLink) there will be 90 hyperlin
        jobLinks = []
        for links in bs.select("td.job_title > a"):
            jobLink = f"https://www.glassdoor.com" + links.get("href")
            jobLinks.append(jobLink)
    
    #scrape job content in jobLinks 
    job_contents = []
    for jobLink in jobLinks: 
        header = {"User-Agent":"Mozilla/5.0 (iPhone; CPU iPhone OS 9_1 like Mac OS X) AppleWebKit/601.1.46 (KHTML, like Gecko) Version/9.0 Mobile/13B143 Safari/601.1"}
        req = urllib.request.Request(jobLink, headers=header) 
        con = urllib.request.urlopen( req ) 
        only_job_content = SoupStrainer("div",id="JobDescriptionContainer")
        soup = BeautifulSoup(con.read(), "html.parser", parse_only=only_job_content)
    #save jobDesc in job_contents
        jobDesc = soup.find("div",class_="jobDescriptionContent")
        time.sleep(2)
        try:
            job_contents.append(jobDesc.get_text())
        except:
            job_contents.append("NA")

    next_page = driver.find_element_by_class_name("next")
    try:
        next_page.find_element_by_tag_name("a")
        next_page.click()
        page.append(1)

    except:
        page_remaining = False
        print("Finished")

HTTPError: HTTP Error 400: Bad Request

In [ ]:
var_list = [jobs_title,companies_name,jobs_location,jobs_salary,companies_star,jobLinks,job_contents,page]
print(list(map(len,var_list)))

In [ ]:
# for ind,val in enumerate(job_contents):
#     if val != "NA":
#         print(ind,"ok")
#     else:
#         print(ind,val)